## Categorical Perception: A Groundwork for Deep Learning
#### Laurent Bonnasse-Gahot & Jean-Pierre Nadal
Python notebook for training a deep convolutional neural network to classify natural images of cats and dogs (using the Kaggle Dogs vs. Cats database), used in Figure 7 (Categorical perception of a cat/dog circular continuum), Section 3.3.1.

In [1]:
import os
import random
import numpy as np
import tensorflow as tf

# for reproducibility
SEED = 1
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [2]:
# bits of code partly taken from the "Image classification from scratch" tutorial
# https://keras.io/examples/vision/image_classification_from_scratch/

# data available at:
# https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

In [3]:
if not os.path.exists('model'):
    os.makedirs('model')

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

#Filter out corrupted images
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [5]:
image_size = (180, 180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [6]:
from tensorflow.keras.layers import Dense, GaussianDropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D

def make_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)  # Apply random data augmentation   
    x = layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d_1')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d_2')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_1')(x)
    x = GaussianDropout(0.1)(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d_3')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d_4')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_2')(x)
    x = GaussianDropout(0.2)(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d_5')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d_6')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_3')(x)  
    x = GlobalAveragePooling2D()(x)
    x = GaussianDropout(0.4)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    return keras.Model(inputs, outputs)

In [7]:
n_trials = 10

epochs = 100

for trial in range(n_trials):
    print('--------------- trial #{} ---------------'.format(trial))
    model = make_model(input_shape=image_size + (3,))

    model.compile(
        optimizer='adam',
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    model.save('model/model_cat_dog_cnn_init_{}.h5'.format(trial))
    model.fit(
        train_ds, epochs=epochs, validation_data=val_ds,
    )
    model.save('model/model_cat_dog_cnn_{}.h5'.format(trial))

--------------- trial #0 ---------------
Epoch 1/100
586/586 [==============================] - 85s 139ms/step - loss: 0.6833 - accuracy: 0.5425 - val_loss: 0.6619 - val_accuracy: 0.6151
Epoch 2/100
586/586 [==============================] - 83s 140ms/step - loss: 0.6586 - accuracy: 0.6043 - val_loss: 0.6147 - val_accuracy: 0.6561
Epoch 3/100
586/586 [==============================] - 83s 140ms/step - loss: 0.6165 - accuracy: 0.6604 - val_loss: 0.5914 - val_accuracy: 0.6766
Epoch 4/100
586/586 [==============================] - 83s 140ms/step - loss: 0.5873 - accuracy: 0.6896 - val_loss: 0.5743 - val_accuracy: 0.7082
Epoch 5/100
586/586 [==============================] - 84s 141ms/step - loss: 0.5648 - accuracy: 0.7084 - val_loss: 0.5398 - val_accuracy: 0.7354
Epoch 6/100
586/586 [==============================] - 81s 138ms/step - loss: 0.5392 - accuracy: 0.7345 - val_loss: 0.5272 - val_accuracy: 0.7364
Epoch 7/100
586/586 [==============================] - 84s 143ms/step - loss: 0.533

586/586 [==============================] - 84s 141ms/step - loss: 0.3475 - accuracy: 0.8495 - val_loss: 0.3042 - val_accuracy: 0.8644
Epoch 12/100
586/586 [==============================] - 84s 143ms/step - loss: 0.3230 - accuracy: 0.8612 - val_loss: 0.2814 - val_accuracy: 0.8765
Epoch 13/100
586/586 [==============================] - 84s 141ms/step - loss: 0.3003 - accuracy: 0.8727 - val_loss: 0.2534 - val_accuracy: 0.8924
Epoch 14/100
586/586 [==============================] - 84s 142ms/step - loss: 0.2899 - accuracy: 0.8803 - val_loss: 0.2211 - val_accuracy: 0.9065
Epoch 15/100
586/586 [==============================] - 84s 143ms/step - loss: 0.2784 - accuracy: 0.8812 - val_loss: 0.2424 - val_accuracy: 0.8983
Epoch 16/100
586/586 [==============================] - 84s 142ms/step - loss: 0.2703 - accuracy: 0.8863 - val_loss: 0.1977 - val_accuracy: 0.9186
Epoch 17/100
586/586 [==============================] - 84s 142ms/step - loss: 0.2537 - accuracy: 0.8935 - val_loss: 0.2128 - val_a

Epoch 67/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1311 - accuracy: 0.9466 - val_loss: 0.1627 - val_accuracy: 0.9361
Epoch 68/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1285 - accuracy: 0.9474 - val_loss: 0.1516 - val_accuracy: 0.9466
Epoch 69/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1287 - accuracy: 0.9462 - val_loss: 0.1396 - val_accuracy: 0.9449
Epoch 70/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1285 - accuracy: 0.9480 - val_loss: 0.1395 - val_accuracy: 0.9462
Epoch 71/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1264 - accuracy: 0.9484 - val_loss: 0.1389 - val_accuracy: 0.9487
Epoch 72/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1339 - accuracy: 0.9444 - val_loss: 0.1443 - val_accuracy: 0.9530
Epoch 73/100
586/586 [==============================] - 84s 141ms/step - loss: 0.1195 - accuracy: 0.9520 - val_loss: 0

Epoch 78/100
586/586 [==============================] - 83s 140ms/step - loss: 0.1194 - accuracy: 0.9522 - val_loss: 0.1528 - val_accuracy: 0.9494
Epoch 79/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1247 - accuracy: 0.9493 - val_loss: 0.1518 - val_accuracy: 0.9470
Epoch 80/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1200 - accuracy: 0.9513 - val_loss: 0.1629 - val_accuracy: 0.9462
Epoch 81/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1176 - accuracy: 0.9526 - val_loss: 0.1372 - val_accuracy: 0.9515
Epoch 82/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1209 - accuracy: 0.9525 - val_loss: 0.1295 - val_accuracy: 0.9483
Epoch 83/100
586/586 [==============================] - 84s 141ms/step - loss: 0.1173 - accuracy: 0.9532 - val_loss: 0.1546 - val_accuracy: 0.9524
Epoch 84/100
586/586 [==============================] - 84s 141ms/step - loss: 0.1176 - accuracy: 0.9529 - val_loss: 0

Epoch 89/100
586/586 [==============================] - 84s 143ms/step - loss: 0.1171 - accuracy: 0.9554 - val_loss: 0.1662 - val_accuracy: 0.9423
Epoch 90/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1136 - accuracy: 0.9563 - val_loss: 0.1397 - val_accuracy: 0.9504
Epoch 91/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1181 - accuracy: 0.9495 - val_loss: 0.1451 - val_accuracy: 0.9515
Epoch 92/100
586/586 [==============================] - 83s 140ms/step - loss: 0.1097 - accuracy: 0.9547 - val_loss: 0.1433 - val_accuracy: 0.9443
Epoch 93/100
586/586 [==============================] - 83s 140ms/step - loss: 0.1114 - accuracy: 0.9538 - val_loss: 0.1362 - val_accuracy: 0.9539
Epoch 94/100
586/586 [==============================] - 84s 141ms/step - loss: 0.1120 - accuracy: 0.9561 - val_loss: 0.1390 - val_accuracy: 0.9526
Epoch 95/100
586/586 [==============================] - 84s 141ms/step - loss: 0.1188 - accuracy: 0.9530 - val_loss: 0

...

586/586 [==============================] - 82s 139ms/step - loss: 0.4810 - accuracy: 0.7751 - val_loss: 0.4048 - val_accuracy: 0.8279
Epoch 11/100
586/586 [==============================] - 83s 141ms/step - loss: 0.4472 - accuracy: 0.7987 - val_loss: 0.4028 - val_accuracy: 0.8272
Epoch 12/100
586/586 [==============================] - 83s 140ms/step - loss: 0.4305 - accuracy: 0.8049 - val_loss: 0.3381 - val_accuracy: 0.8571
Epoch 13/100
586/586 [==============================] - 83s 141ms/step - loss: 0.4043 - accuracy: 0.8238 - val_loss: 0.3374 - val_accuracy: 0.8541
Epoch 14/100
586/586 [==============================] - 83s 140ms/step - loss: 0.3752 - accuracy: 0.8371 - val_loss: 0.3175 - val_accuracy: 0.8727
Epoch 15/100
586/586 [==============================] - 84s 142ms/step - loss: 0.3575 - accuracy: 0.8437 - val_loss: 0.3153 - val_accuracy: 0.8648
Epoch 16/100
586/586 [==============================] - 83s 141ms/step - loss: 0.3434 - accuracy: 0.8542 - val_loss: 0.2554 - val_a

Epoch 66/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1472 - accuracy: 0.9423 - val_loss: 0.1708 - val_accuracy: 0.9361
Epoch 67/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1545 - accuracy: 0.9355 - val_loss: 0.1663 - val_accuracy: 0.9385
Epoch 68/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1470 - accuracy: 0.9402 - val_loss: 0.1541 - val_accuracy: 0.9413
Epoch 69/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1479 - accuracy: 0.9395 - val_loss: 0.1909 - val_accuracy: 0.9434
Epoch 70/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1543 - accuracy: 0.9387 - val_loss: 0.1927 - val_accuracy: 0.9361
Epoch 71/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1549 - accuracy: 0.9374 - val_loss: 0.1390 - val_accuracy: 0.9462
Epoch 72/100
586/586 [==============================] - 84s 141ms/step - loss: 0.1520 - accuracy: 0.9372 - val_loss: 0

Epoch 77/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1135 - accuracy: 0.9553 - val_loss: 0.1622 - val_accuracy: 0.9485
Epoch 78/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1164 - accuracy: 0.9558 - val_loss: 0.1376 - val_accuracy: 0.9528
Epoch 79/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1133 - accuracy: 0.9566 - val_loss: 0.1442 - val_accuracy: 0.9436
Epoch 80/100
586/586 [==============================] - 84s 143ms/step - loss: 0.1191 - accuracy: 0.9530 - val_loss: 0.1377 - val_accuracy: 0.9511
Epoch 81/100
586/586 [==============================] - 84s 143ms/step - loss: 0.1076 - accuracy: 0.9574 - val_loss: 0.1350 - val_accuracy: 0.9481
Epoch 82/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1097 - accuracy: 0.9546 - val_loss: 0.1242 - val_accuracy: 0.9522
Epoch 83/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1128 - accuracy: 0.9541 - val_loss: 0

Epoch 88/100
586/586 [==============================] - 84s 143ms/step - loss: 0.1208 - accuracy: 0.9504 - val_loss: 0.1601 - val_accuracy: 0.9494
Epoch 89/100
586/586 [==============================] - 84s 142ms/step - loss: 0.1293 - accuracy: 0.9466 - val_loss: 0.1471 - val_accuracy: 0.9468
Epoch 90/100
586/586 [==============================] - 84s 143ms/step - loss: 0.1229 - accuracy: 0.9510 - val_loss: 0.1518 - val_accuracy: 0.9494
Epoch 91/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1194 - accuracy: 0.9507 - val_loss: 0.1506 - val_accuracy: 0.9417
Epoch 92/100
586/586 [==============================] - 84s 143ms/step - loss: 0.1186 - accuracy: 0.9520 - val_loss: 0.1737 - val_accuracy: 0.9393
Epoch 93/100
586/586 [==============================] - 84s 143ms/step - loss: 0.1221 - accuracy: 0.9501 - val_loss: 0.1466 - val_accuracy: 0.9447
Epoch 94/100
586/586 [==============================] - 84s 143ms/step - loss: 0.1180 - accuracy: 0.9522 - val_loss: 0

Epoch 99/100
586/586 [==============================] - 83s 141ms/step - loss: 0.1045 - accuracy: 0.9588 - val_loss: 0.1422 - val_accuracy: 0.9517
Epoch 100/100
586/586 [==============================] - 83s 140ms/step - loss: 0.1116 - accuracy: 0.9538 - val_loss: 0.1348 - val_accuracy: 0.9517
